# build-metro-application

Builds metro appliction that accepts Kafka messages from the edge(s). 

Aggregates/Analyze messages 
- push to storage
- generate events 
- send notifcations
- deep analysis
- host views to monitor the 'goings on'.

In [ ]:
!pip install streamsx.eventstreams

In [ ]:
import urllib3
import time
import json
import os
import sys
import collections
import warnings

from streamsx.topology.topology import Topology
from streamsx.topology.schema import CommonSchema
from streamsx.topology.context import submit, ContextTypes
import streamsx.eventstreams as eventstreams

from streamsx.rest_primitives import Instance
from streamsx.topology import context
import streamsx.rest as rest


In [ ]:
urllib3.disable_warnings()
# Cell to grab Streams instance config object and REST reference
from icpd_core import icpd_util
STREAMS_INSTANCE_NAME = "edge"
streams_cfg=icpd_util.get_service_instance_details(name=STREAMS_INSTANCE_NAME)
streams_cfg[context.ConfigParams.SSL_VERIFY] = False
streams_instance = Instance.of_service(streams_cfg)

In [ ]:
## Define the EventStreams topics to access 
EVENTSTREAMS_TOPIC = 'DefaultTopic'

# The eventstreams group id to use as a base
GROUP_NAME_BASE = 'MetroEdge-'

In [ ]:
# Enter in your Eventstreams credentials as JSON
import getpass
eventstreams_credentials_json = getpass.getpass('Your Event Streams credentials:')
app_config_name = eventstreams.configure_connection(streams_instance, name='eventstreams', credentials=eventstreams_credentials_json)


In [ ]:
class SlideWindow(object):
    """ Window with slide_length elements. 
    
    Window fills, intial output will have less than slide_length.
    
    Args:
        slide_length: maximum number of elements in window.
        
    Returns:
        list of up to 25 of the last tups input.
    """
    def __init__(self, slide_length:int=25):
        self.slide_length = slide_length

    def __enter__(self):
        self.chunk = collections.deque(maxlen=self.slide_length)
        
    def __exit__(self, exc_type, exc_value, traceback):
        # __enter__ and __exit__ must both be defined.
        pass
    
    def __call__(self, tup) -> list:
        self.chunk.append(tup)
        return list(self.chunk)
        

In [ ]:
urllib3.disable_warnings()
def build_metro() -> Topology:
    """ metro application subscribing to two topics 
    
    * Subscribed topics are reflected out view.
    * Metrics are windowed
    
    Returns:
        Topology of the application. 
    """
    topo = Topology('EdgeMetroSubscribe')
    # Subscribe to same topic as a stream 

    # All items in the feed
    combined_feed = eventstreams.subscribe(topo, schema=CommonSchema.Json, topic=EVENTSTREAMS_TOPIC, group=GROUP_NAME_BASE + EVENTSTREAMS_TOPIC, credentials=app_config_name)
    from_evstr1 = combined_feed.filter(lambda t: "camera_metrics" in t)

    # collect Collect Metrics
    from_evstr1.view(name="ClassificationMetrics")
    from_evstr1.print(name="classificationPrint")

    # window the Metrics
    windowSlide = from_evstr1.map(SlideWindow())
    windowSlide.view(name="WindowUncertain")
    windowSlide.print(name="windowPrint")

    # Collect the uncertain predictions
    from_evstr2 = combined_feed.filter(lambda t: "image" in t)
    from_evstr2.view(name="UncertainPredictions")
    from_evstr2.print(name="uncertainPrint")

    return topo

# Generate the topology
topo = build_metro()

# Cancel the job from the instance if it is already running...
for job in streams_instance.get_jobs():
    if job.name == topo.name:
        print("Cancelling old job:", job.name)
        job.cancel()
    
# Setup the job config
job_config = context.JobConfig(job_name = topo.name, tracing = "debug")
job_config.add(streams_cfg)
    
# Actually submit the job
print("Building and submitting new job:", topo.name)
submission_result = context.submit('DISTRIBUTED', topo, streams_cfg)

if submission_result.return_code == 0:
    print("Job built and submitted successfully.")
    print("  Job ID:",submission_result.jobId)


## Monitor from metro. 

Once the metro is up and running the [render-metro-views.jupyter-py36.ipynb](render-metro-views.jupyter-py36.ipynb) notebook can monitor and enhance the processing.